# 1. Import

In [ ]:
import numpy as np
import re
from os import listdir
from os.path import isdir, join, islink

In [ ]:
mypath = 'logs/'
startswith = input("startwith filter? ")
logs = [f for f in listdir(mypath) if (isdir(join(mypath, f)) or islink(join(mypath, f))) and f.startswith(startswith)]

In [ ]:
logs = [log for log in logs if input(log) == ""]
print(logs)

In [ ]:
logs

In [ ]:
logs = [
        'comp_end_final_.2018.07.26.14.31',
 'comp_ru_final_.2018.07.26.14.17',
 'comp_sp1_final_.2018.07.27.17.19',
    'comp_ch_final_.2018.07.26.15.33',
]


In [ ]:
lognames = {
    'comp_ch_final_.2018.07.26.15.33': "ch_final",
    'comp_end_1_.2018.07.25.17.29': "end_1",
    'comp_end_1_.2018.07.25.17.40': "end_1",
    'comp_end_1_.2018.07.25.18.07': "end_1",
    'comp_end_1_.2018.07.25.18.16': "end_1",
    'comp_end_final_.2018.07.26.14.31': "end_final",
    'comp_end_with_small_weights_.2018.07.25.22.14': "end_with_small_weights",
    'comp_ru_final_.2018.07.26.14.17': "ru_final",
    'comp_sp1__.2018.07.26.12.05': "sp1",
    'comp_sp1__minusOneTozeros_.2018.07.26.13.06': "sp1__minusOneTozeros",
    'comp_sp1_final_.2018.07.27.17.19': "sp1_final",
    'comp_sp1_withoutMasking_.2018.07.26.13.25': "sp1_withoutMasking",
 }


In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


event_acc = {log: EventAccumulator('logs/'+log) for log in logs}
for log in logs:
    event_acc[log].Reload()
# Show all tags in the log file
# print(event_acc.Tags())

# E. g. get wall clock, number of steps and value for a scalar 'Accuracy'
# w_times, step_nums, vals = zip(*event_acc.Scalars('QAperson_acc'))
# print(w_times)
# print(step_nums)
# print(vals)

In [ ]:
histories = {log: {k:[x.value for x in event_acc[log].scalars.Items(k)]  for k in event_acc[log].scalars.Keys()} for log in logs}

# 2. Plots

### model overall loss

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(history["loss"])
plt.plot(history['val_loss'])
plt.plot(history['test_loss'])
plt.title('model overall loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'test'], loc='upper right')
# plt.savefig('plots/model_overall_loss' +
#             datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')
# plt.close(fig)

plt.show()


# In[471]:

### model avaerage accuracy

In [ ]:
%config InlineBackend.figure_format = 'retina'
# summarize history for loss
splits = ['train', 'val', 'test']
# splits = ['train']

# print(experminets["dts"])
for split in splits:
    legends = []
    fig = plt.figure()
    for log in logs:
        if log not in logs:
            continue
        logname = lognames[log] if log in lognames else log
#         logname = re.sub('.*_','',log)
#         logname = int(round(1-float("."+re.sub('\+.*','',logname)),2)*100)
    #     logname = log
        history = histories[log]
        plt.plot(history[split+"_acc"])
        legends.append(str(logname) + "% = "+ str(round(history[split + '_acc'][-1]*100,2)))
    plt.title('model average accuracy of '+split+" dataset")
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(legends, loc='lower right')
    # plt.savefig('plots/model_average_accuracy' +
    #             datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')
    plt.show()
    plt.close(fig)

In [ ]:
%config InlineBackend.figure_format = 'retina'
# summarize history for loss
starts_with = {
    "train": "QA",
    "val": "val_",
    "test": "test_",
}
splits = ['train', 'val', 'test']

experminets = {
    "dts" : [log for log in logs if log.startswith("diff_train_sizes")], #diff_train_sizes
}

# print(experminets["dts"])
for split in splits:
    ind = None
    legends = []
    fig = plt.figure()
    plts = {}
    for log in logs:
#         if log not in experminets["dts"]:
#             continue
        logname = lognames[log] if log in lognames else log
#         logname = re.sub('.*_','',log)
#         logname = round(1-float("."+re.sub('\+.*','',logname)),2)
        history = histories[log]
        if ind is None:
            selected = sorted([x for x in history 
                               if x.endswith("_acc") and
                               x.startswith(starts_with[split]) and
                               not x.endswith("QAutf8_acc") and
                               x != starts_with[split]+"acc"],reverse=True)
            ind = range(len(selected))
        data = [history[x][-1] for x in selected]
        legends.append(logname)
        plts[log] = plt.bar(ind, data, 0.35)
        
#     print(selected)
    plt.xticks(ind, [re.sub('.*_','',x.replace("_acc","")).replace("QA","") for x in selected],rotation=90)
#     plt.yticks(np.arange(80, 100)/100)
#     plt.legend((p1[0], p2[0]), ('Men', 'Women'))
    plt.gca().set_ylim([0.93,1])
    plt.title('sample-based model accuracy of the four approaches('+split+")")
    plt.ylabel('accuracy')
#     plt.xlabel('epoch')
    plt.legend(legends, loc='lower right')
    # plt.savefig('plots/model_average_accuracy' +
    #             datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')
    plt.show()
    plt.close(fig)

In [ ]:
%config InlineBackend.figure_format = 'retina'
# summarize history for loss
splits = ['train', 'val', 'test']
# splits = ['train']

# print(experminets["dts"])
for split in splits:
    legends = []
    fig = plt.figure()
    for log in logs:
#         if log not in [log for log in logs 
#                        if log.startswith("emb_") or 
#                        log.startswith("only_emb") or 
#                        log.startswith("diff_train_sizes_1") or 
#                        log.startswith("diff_train_sizes_8")]:
#             continue
        logname = lognames[log] if log in lognames else log
#         logname = re.sub('\+.*','',log)
#         logname = logname.replace("diff_train_sizes","baseline").replace("_1","_large").replace("_8","_small")
#         logname = int(round(1-float("."+re.sub('\+.*','',logname)),2)*100)
#         logname = log
        history = histories[log]
        if (split + '_acc') not in history:
            print(history)
            continue
        plt.plot(history[split+"_acc"][:30])
        legends.append(str(logname) + " = "+ str(round(history[split + '_acc'][-1]*100,2)))
    plt.title('model average accuracy of '+split+" dataset")
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(legends, loc='lower right')
    # plt.savefig('plots/model_average_accuracy' +
    #             datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')
    plt.show()
    plt.close(fig)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# summarize history for loss
plt.plot(history["loss"])
plt.plot(history['val_loss'])
plt.plot(history['test_loss'])
plt.title('model overall loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'test'], loc='upper right')
plt.savefig('plots/model_overall_loss' +
            datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')

# plt.show()

In [ ]:
# summarize history for loss
plt.plot(history["train_acc"])
plt.plot(history['val_acc'])
plt.plot(history['test_acc'])
plt.title('model average accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend([x +" = "+ str(round(history[x+'_acc'][-1]*100,2)) for x in ['train', 'val', 'test']], loc='lower right')
# for i, x in enumerate(['train', 'val', 'test']):
#     plt.annotate(str(round(history[x+'_acc'][-1]*100,2)),
#                  xy=(len(history[x+'_acc']), history[x+'_acc'][-1]), 
#                  textcoords='figure pixels', 
#                  xytext=(-20,-10))
plt.savefig('plots/model_average_accuracy' +
            datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')

plt.show()

In [ ]:
# VALIDATAION = True
# prefix = "val_" if VALIDATAION else ""
# del prefix
legends = []
for x in model.output_names:
    # summarize history for accuracy
    plt.plot(history[x+"_acc"])
    legends.append(""+x +" = "+ str(round(history[x+'_acc'][-1]*100,2)))
#     plt.plot(history[x+"_acc"])
#     legends.append("val_"+x)
#     plt.plot(history["val_" + x+"_acc"])
#     legends.append("train_"+x)
plt.title('model indiviual accuracy on test dataset')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(legends, loc='lower right')
plt.savefig('plots/accuracy_all' +
            datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

legends = []
for x in model.output_names:
    # summarize history for loss
    plt.plot(history["test_"+x+"_loss"])
    legends.append(""+x +" = "+ str(round(history[x+'_loss'][-1]*100,2)))
plt.title('model individual loss on test dataset')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(legends, loc='upper right')
plt.savefig('plots/loss_all' +
            datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M") + '.png')
plt.show()

# 10.2 Pie

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re
import math

# Fixing random state for reproducibility
np.random.seed(19680801)

experiments = [log for log in logs if log.startswith("diff_taggers")]
# experiments.sort() # sorts normally by alphabetical order
experiments.sort(key=len, reverse=True) # sorts by descending length
# print(experiments)
alltools = ["","MX","ST","FA","AM"]
cmaps = ['Grey', 'Purple', 'Blue', 'Green', 'Orange', 'Red',]
experiments
widths = []
thetas = []
radii = []
colors = []
# print(experiments)
for log in experiments:
    logname = re.sub('.*_','',log)
    logname = re.sub('\+.*','',logname)
    tools = logname.split(",")
    logname = "s_"+logname if "small" in log else "l_"+logname
    issmall = "small" in log
    if not issmall:
        continue
    history = histories[log]
    history = history["test_acc"][-1]
    width = np.pi * 8 / (math.pow(4,len(tools)))
    theta = [alltools.index(t) for t in tools] + [0] * (len(alltools) - len(tools))
    print(tools, width, theta, history)
    theta = [90*(t-1)/math.pow(4,0 if i == 0 else i-1) for i, t in enumerate(theta)]
#     print(len(tools), np.pi * 2 / (math.pow(4,len(tools))))
    theta = sum(theta) + width /2
    print(tools, theta)
    widths.append(width)
    thetas.append(theta)
    radii.append(history)
    colors.append(alltools.index(tools[-1]))
#     print(issmall,tools, "=", history, theta)


# radii = [(1 - r) * 100 for r in radii]
radii = [r - min(radii)  for r in radii]
# radii

# Compute pie slices



ax = plt.subplot(111, projection='polar')
bars = ax.bar(thetas, radii, width=widths, bottom=0.0)

# Use custom colors and opacity
for r, bar in zip(colors, bars):
    bar.set_facecolor(cmaps[r])
    bar.set_alpha(1)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re
import math

# Fixing random state for reproducibility
np.random.seed(19680801)

experiments = [log for log in logs if log.startswith("diff_taggers")]
# experiments.sort() # sorts normally by alphabetical order
experiments.sort(key=len, reverse=True) # sorts by descending length
# print(experiments)
alltools = ["","MX","ST","FA","AM"]
cmaps = ['Grey', 'Purple', 'Blue', 'Green', 'Orange', 'Red',]
# experiments

In [ ]:
all = {}
SMALL = True
METRIC = "test_acc"#"val_QAutf8_acc"
# print(experiments)
for log in experiments:
    logname = re.sub('.*_','',log)
    logname = re.sub('\+.*','',logname)
    tools = logname.split(",")
    issmall = "small" in log
    if SMALL != issmall:
        continue
    logname = "s_"+logname if issmall else "l_"+logname
    history = histories[log]
    all[",".join(sorted(tools))] = history[METRIC][29]

all

In [ ]:
import itertools
accuracies = {}
# for L in range(0, len(tools)+1):
L = len(tools) 
for subset in itertools.combinations(alltools[1:], L):
    for p in itertools.permutations(subset):
        accuracies[p]=[]
        for e in range(L+1):
            accuracies[p].append(all[",".join(sorted(p[:e]))])
            
accuracies

In [ ]:
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10,10)

print("range", [min([all[x] for x in all]),max([all[x] for x in all])])
ind = range(len(accuracies))
for ii in reversed(range(L+1)):
    if ii != 0: 
        colors = [cmaps[alltools.index(i[ii-1])] for i in accuracies] 
    else: 
        colors = ["Grey" for i in accuracies]
    plt.bar(ind, [accuracies[i][ii] for i in accuracies], color=colors)
#     print(selected)
plt.xticks(ind, ["-".join(i) for i in accuracies],rotation=90)
plt.gca().set_ylim([min([all[x] for x in all])-0.002,max([all[x] for x in all])+0.002])
plt.title('model accuracy of multiple combinations of tools on '+ ('small' if SMALL else 'large' )+' dataset ('+METRIC+')')
plt.ylabel('accuracy')
plt.legend(list(reversed(alltools)), loc='lower right')
plt.show()
plt.close(fig)

In [ ]:
import itertools
tools = ["MX", "FA", "AM", "ST"]
all = {}
SMALL = False
METRIC = "test_acc"#"val_QAutf8_acc"
for SMALL in [False, True]:
    for METRIC in ["test_acc","val_QAutf8_acc", "val_QApos_acc"]:
        # print(experiments)
        for log in experiments:
            logname = re.sub('.*_','',log)
            logname = re.sub('\+.*','',logname)
            tools = logname.split(",")
            issmall = "small" in log
            if SMALL != issmall:
                continue
            logname = "s_"+logname if issmall else "l_"+logname
            history = histories[log]
            all[",".join(sorted(tools))] = history[METRIC][29]

        accuracies = {}
        # for L in range(0, len(tools)+1):
        L = len(tools) 
        for subset in itertools.combinations(tools, L):
            for p in itertools.permutations(subset):
                accuracies[p]=[]
                for e in range(L+1):
                    accuracies[p].append(all[",".join(sorted(p[:e]))])

        %config InlineBackend.figure_format = 'retina'
        plt.rcParams['figure.figsize'] = (10,10)

        ind = range(len(accuracies))
        for ii in reversed(range(L+1)):
            if ii != 0: 
                colors = [cmaps[alltools.index(i[ii-1])] for i in accuracies] 
            else: 
                colors = ["Grey" for i in accuracies]
            plt.bar(ind, [accuracies[i][ii] for i in accuracies], color=colors)
        #     print(selected)
        plt.xticks(ind, ["-".join(i) for i in accuracies],rotation=90)
        plt.gca().set_ylim([min([all[x] for x in all])-0.002,max([all[x] for x in all])+0.002])
        plt.title('model accuracy of multiple combinations of tools on '+ ('small' if SMALL else 'large' )+' dataset ('+METRIC+')')
        plt.ylabel('accuracy')
        plt.legend(list(reversed(alltools)), loc='lower right')
        plt.show()
        plt.close(fig)